In [3]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "847b322c",
   "metadata": {},
   "source": [
    "# CA684 Machine Learning Assignment Spring 2023\n",
    "\n",
    "Dublin City University has teamed up with **Etsy** to create the 2023 CA684 Machine Learning assignment.\n",
    "\n",
    "## Etsy.com\n",
    "\n",
    "Etsy is the global marketplace for unique and creative goods. It’s home to a universe of special, extraordinary items, from unique handcrafted pieces to vintage treasures. Etsy operates two-sided online marketplaces that collectively connect nearly 100 million passionate and creative buyers and 7.7 million sellers around the world. \n",
    "\n",
    "Buyers come to the Etsy marketplace for meaningful, one-of-a-kind items handcrafted and curated with passion and expertise by our creative entrepreneurs. We are focused on attracting potential buyers to Etsy for everyday items that have meaning and those “special” purchase occasions that happen throughout the year. These include items that reflect an individual’s unique style; gifting that demonstrates thought and care; and celebrations that express creativity and fun.\n",
    "\n",
    "On Etsy, someone searches for …\n",
    "- Wall art every two seconds\n",
    "- Wedding-related items every two seconds\n",
    "- Gift-related items every second\n",
    "- Antique or vintage items every half a second\n",
    "\n",
    "## Challenge\n",
    "\n",
    "Etsy has nearly 100 Million active listings on the etsy.com marketplace for sale from more than 5 million active sellers. You are provided a subset of those products as the training data. \n",
    "\n",
    "The task is to leverage the training dataset to learn patterns from and to predict the following attributes on an unseen test dataset given a products’ information:\n",
    "\n",
    "- top category id\n",
    "- bottom category id\n",
    "- color id\n",
    "\n",
    "The goal is to maximize F1 for each of the classes on each attribute to predict (top category, bottom category and color). Your approaches and models will be benchmarked against a hidden test dataset.\n",
    "\n",
    "Bonus points will be given for submissions that:\n",
    "\n",
    "- the trained model predicts the three attributes at the same time\n",
    "- visualize some learned representations or embeddings and show that similar items cluster together\n",
    "- compares the performance of pre-trained embeddings taken from hubs or papers with a fine-tuned model\n",
    "\n",
    "\n",
    "## Getting Started\n",
    "\n",
    "Here is some sample code to get you started on the challenge!\n",
    "\n",
    "Happy Hacking!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "49b9b3bb",
   "metadata": {},
   "outputs": [],
   "source": [
    "# libraries\n",
    "import io\n",
    "import os\n",
    "from datetime import datetime\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import tensorflow as tf\n",
    "import matplotlib.pyplot as plt\n",
    "import matplotlib.image as mpimg\n",
    "from PIL import Image\n",
    "from tensorflow.keras.applications.inception_v3 import InceptionV3\n",
    "from tensorflow.keras.preprocessing import image\n",
    "from tensorflow.keras.models import Model\n",
    "from tensorflow.keras.layers import Dense, GlobalAveragePooling2D\n",
    "from sklearn import metrics\n",
    "\n",
    "# Matplotlib configuration\n",
    "font = { 'family': 'DejaVu Sans', 'weight': 'bold', 'size': 16 }\n",
    "plt.rc('font', **font)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "0d1373e8",
   "metadata": {},
   "outputs": [],
   "source": [
    "# set random seed\n",
    "np.random.seed(seed=42)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "34d263ed",
   "metadata": {},
   "source": [
    "## Dataset\n",
    "\n",
    "The train and test datasets are shared in two different formats, parquet and TFRecords, for you to choose which one is more appropriate for your approaches.\n",
    "\n",
    "In order to download the dataset, please sign the attached NDA and email it to dazcona@etsy.com with copy to legal@etsy.com from your DCU email address using the following subject: *Machine Learning University Challenge Spring 2023 <your DCU username>*\n",
    "\n",
    "**Note** that images are only in the TFRecords!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "36186dd2",
   "metadata": {},
   "outputs": [],
   "source": [
    "PATH = f\"/data/\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "ec773a64",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "parquet  tfrecords\r\n"
     ]
    }
   ],
   "source": [
    "!ls {PATH}"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "a500b4d8",
   "metadata": {},
   "outputs": [],
   "source": [
    "parquet_filenames_train = !ls {PATH}/parquet/train"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "1f5004a7",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['part-00000-of-00278.parquet',\n",
       " 'part-00001-of-00278.parquet',\n",
       " 'part-00002-of-00278.parquet']"
      ]
     },
     "execution_count": 6,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "parquet_filenames_train[:3]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "6b146499",
   "metadata": {},
   "outputs": [],
   "source": [
    "parquet_filenames_test = !ls {PATH}/parquet/test"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "id": "7c169b61",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['part-00000-of-00278.parquet',\n",
       " 'part-00001-of-00278.parquet',\n",
       " 'part-00002-of-00278.parquet']"
      ]
     },
     "execution_count": 8,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "parquet_filenames_test[:3]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 9,
   "id": "be7b1870",
   "metadata": {},
   "outputs": [],
   "source": [
    "tfrecord_filenames_train = !ls {PATH}/tfrecords/train"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 10,
   "id": "67139754",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['part-00000-of-00278.tfrecord',\n",
       " 'part-00001-of-00278.tfrecord',\n",
       " 'part-00002-of-00278.tfrecord']"
      ]
     },
     "execution_count": 10,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "tfrecord_filenames_train[:3]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 11,
   "id": "e8afe7da",
   "metadata": {},
   "outputs": [],
   "source": [
    "tfrecord_filenames_test = !ls {PATH}/tfrecords/test"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 12,
   "id": "7dc1efdd",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['part-00000-of-00278.tfrecord',\n",
       " 'part-00001-of-00278.tfrecord',\n",
       " 'part-00002-of-00278.tfrecord']"
      ]
     },
     "execution_count": 12,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "tfrecord_filenames_test[:3]"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "cc5e1ae4",
   "metadata": {},
   "source": [
    "## Exploratory Data Analysis\n",
    "\n",
    "It is important to familiarize yourself with the dataset by using measures of centrality (e.g. mean) and statistical dispersion (e.g. variance) and data visualization methods. The following is just some Pandas preprocessing and Matplotlib visualizations to get you started. Feel free to explore the data much further and come up with ideas that might help you in the task!"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "478f0fa6",
   "metadata": {},
   "source": [
    "### Training Dataset\n",
    "\n",
    "This dataset can be used to train models and learn patterns from.\n",
    "\n",
    "#### Parquet"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 13,
   "id": "cef20992",
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_parquet_data(folder, N):\n",
    "    \"\"\" Read parquet data from mutiple files \"\"\"\n",
    "\n",
    "    filenames = [ \n",
    "        f\"{PATH}/parquet/{folder}/part-{i:05d}-of-{N:05d}.parquet\" \n",
    "        for i in range(N)\n",
    "    ]\n",
    "\n",
    "    dataframes = [ \n",
    "        pd.read_parquet(filename) \n",
    "        for filename in filenames\n",
    "    ]\n",
    "    \n",
    "    return pd.concat(dataframes)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 14,
   "id": "fd364441",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_train = get_parquet_data(\"train\", len(parquet_filenames_train))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 15,
   "id": "3a5df6fa",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of rows in train: 245,485'"
      ]
     },
     "execution_count": 15,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of rows in train: {len(df_train):,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 16,
   "id": "72ff1336",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of unique products in train: 245,485'"
      ]
     },
     "execution_count": 16,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of unique products in train: {len(df_train['product_id'].unique()):,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 17,
   "id": "a7167f76",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of columns in train: 21'"
      ]
     },
     "execution_count": 17,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of columns in train: {len(df_train.columns):,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 18,
   "id": "24ac7144",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['product_id',\n",
       " 'title',\n",
       " 'description',\n",
       " 'tags',\n",
       " 'type',\n",
       " 'room',\n",
       " 'craft_type',\n",
       " 'recipient',\n",
       " 'material',\n",
       " 'occasion',\n",
       " 'holiday',\n",
       " 'art_subject',\n",
       " 'style',\n",
       " 'shape',\n",
       " 'pattern',\n",
       " 'bottom_category_id',\n",
       " 'bottom_category_text',\n",
       " 'top_category_id',\n",
       " 'top_category_text',\n",
       " 'color_id',\n",
       " 'color_text']"
      ]
     },
     "execution_count": 18,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "list(df_train.columns)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 19,
   "id": "8999100d",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "product_id                                                      543497833\n",
       "title                   Full-face custom motorcycle helmet ,Motorcycle...\n",
       "description             Helmetartthai from Thailand ( Since 2016) \\n\\n...\n",
       "tags                    predator helmet,motorcycle helmet,helmet,handm...\n",
       "type                                                             physical\n",
       "room                                                                 None\n",
       "craft_type                                                           None\n",
       "recipient                                                            None\n",
       "material                                                             None\n",
       "occasion                                                             None\n",
       "holiday                                                              None\n",
       "art_subject                                                          None\n",
       "style                                                                None\n",
       "shape                                                                None\n",
       "pattern                                                              None\n",
       "bottom_category_id                                                   2804\n",
       "bottom_category_text    accessories.hats_and_caps.helmets.sports_helme...\n",
       "top_category_id                                                         0\n",
       "top_category_text                                             accessories\n",
       "color_id                                                               12\n",
       "color_text                                                         purple\n",
       "Name: 0, dtype: object"
      ]
     },
     "execution_count": 19,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_train.iloc[0]"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 20,
   "id": "c35c9c94",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<div>\n",
       "<style scoped>\n",
       "    .dataframe tbody tr th:only-of-type {\n",
       "        vertical-align: middle;\n",
       "    }\n",
       "\n",
       "    .dataframe tbody tr th {\n",
       "        vertical-align: top;\n",
       "    }\n",
       "\n",
       "    .dataframe thead th {\n",
       "        text-align: right;\n",
       "    }\n",
       "</style>\n",
       "<table border=\"1\" class=\"dataframe\">\n",
       "  <thead>\n",
       "    <tr style=\"text-align: right;\">\n",
       "      <th></th>\n",
       "      <th>product_id</th>\n",
       "      <th>title</th>\n",
       "      <th>description</th>\n",
       "      <th>tags</th>\n",
       "      <th>type</th>\n",
       "      <th>room</th>\n",
       "      <th>craft_type</th>\n",
       "      <th>recipient</th>\n",
       "      <th>material</th>\n",
       "      <th>occasion</th>\n",
       "      <th>...</th>\n",
       "      <th>art_subject</th>\n",
       "      <th>style</th>\n",
       "      <th>shape</th>\n",
       "      <th>pattern</th>\n",
       "      <th>bottom_category_id</th>\n",
       "      <th>bottom_category_text</th>\n",
       "      <th>top_category_id</th>\n",
       "      <th>top_category_text</th>\n",
       "      <th>color_id</th>\n",
       "      <th>color_text</th>\n",
       "    </tr>\n",
       "  </thead>\n",
       "  <tbody>\n",
       "    <tr>\n",
       "      <th>0</th>\n",
       "      <td>543497833</td>\n",
       "      <td>Full-face custom motorcycle helmet ,Motorcycle...</td>\n",
       "      <td>Helmetartthai from Thailand ( Since 2016) \\n\\n...</td>\n",
       "      <td>predator helmet,motorcycle helmet,helmet,handm...</td>\n",
       "      <td>physical</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>...</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>2804</td>\n",
       "      <td>accessories.hats_and_caps.helmets.sports_helme...</td>\n",
       "      <td>0</td>\n",
       "      <td>accessories</td>\n",
       "      <td>12</td>\n",
       "      <td>purple</td>\n",
       "    </tr>\n",
       "    <tr>\n",
       "      <th>1</th>\n",
       "      <td>718680498</td>\n",
       "      <td>Full-face predator motorcycle  helmet ,Custom ...</td>\n",
       "      <td>Helmetartthai from Thailand ( Since 2016) \\n\\n...</td>\n",
       "      <td>Predator helmet,Custom helmet,Handmade helme,P...</td>\n",
       "      <td>physical</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>...</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>None</td>\n",
       "      <td>2804</td>\n",
       "      <td>accessories.hats_and_caps.helmets.sports_helme...</td>\n",
       "      <td>0</td>\n",
       "      <td>accessories</td>\n",
       "      <td>1</td>\n",
       "      <td>black</td>\n",
       "    </tr>\n",
       "  </tbody>\n",
       "</table>\n",
       "<p>2 rows × 21 columns</p>\n",
       "</div>"
      ],
      "text/plain": [
       "   product_id                                              title  \\\n",
       "0   543497833  Full-face custom motorcycle helmet ,Motorcycle...   \n",
       "1   718680498  Full-face predator motorcycle  helmet ,Custom ...   \n",
       "\n",
       "                                         description  \\\n",
       "0  Helmetartthai from Thailand ( Since 2016) \\n\\n...   \n",
       "1  Helmetartthai from Thailand ( Since 2016) \\n\\n...   \n",
       "\n",
       "                                                tags      type  room  \\\n",
       "0  predator helmet,motorcycle helmet,helmet,handm...  physical  None   \n",
       "1  Predator helmet,Custom helmet,Handmade helme,P...  physical  None   \n",
       "\n",
       "  craft_type recipient material occasion  ... art_subject style shape pattern  \\\n",
       "0       None      None     None     None  ...        None  None  None    None   \n",
       "1       None      None     None     None  ...        None  None  None    None   \n",
       "\n",
       "  bottom_category_id                               bottom_category_text  \\\n",
       "0               2804  accessories.hats_and_caps.helmets.sports_helme...   \n",
       "1               2804  accessories.hats_and_caps.helmets.sports_helme...   \n",
       "\n",
       "  top_category_id  top_category_text color_id  color_text  \n",
       "0               0        accessories       12      purple  \n",
       "1               0        accessories        1       black  \n",
       "\n",
       "[2 rows x 21 columns]"
      ]
     },
     "execution_count": 20,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_train.head(2)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 21,
   "id": "cc00c799",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array(['physical', None, 'download'], dtype=object)"
      ]
     },
     "execution_count": 21,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_train['type'].unique()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 22,
   "id": "dd36fe38",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "physical    236137\n",
       "download      8074\n",
       "None          1274\n",
       "Name: type, dtype: int64"
      ]
     },
     "execution_count": 22,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_train['type'].value_counts(dropna=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 23,
   "id": "2ba5a8ea",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of colors in train: 20'"
      ]
     },
     "execution_count": 23,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "COLORS = df_train['color_id'].unique()\n",
    "NUM_COLORS = len(COLORS)\n",
    "f\"Number of colors in train: {NUM_COLORS:,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 24,
   "id": "1c585195",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of top categories in train: 15'"
      ]
     },
     "execution_count": 24,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of top categories in train: {len(df_train['top_category_text'].unique()):,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 25,
   "id": "5d67aa45",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "home_and_living                53200\n",
       "craft_supplies_and_tools       42348\n",
       "clothing                       32028\n",
       "toys_and_games                 14427\n",
       "accessories                    12610\n",
       "bath_and_beauty                12426\n",
       "paper_and_party_supplies       10811\n",
       "shoes                          10355\n",
       "weddings                       10333\n",
       "jewelry                         9457\n",
       "art_and_collectibles            8741\n",
       "electronics_and_accessories     8177\n",
       "books_movies_and_music          7378\n",
       "bags_and_purses                 6645\n",
       "pet_supplies                    6549\n",
       "Name: top_category_text, dtype: int64"
      ]
     },
     "execution_count": 25,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "pd.value_counts(df_train['top_category_text'], sort=True, ascending=False)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 26,
   "id": "aa6961db",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 1500x600 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "figsize = (15, 6)\n",
    "column = 'top_category_text'\n",
    "x_limit = 60000\n",
    "x_step = 10000\n",
    "color = 'darkcyan'\n",
    "    \n",
    "figure, ax = plt.subplots(figsize=figsize)\n",
    "pd.value_counts(\n",
    "    df_train[column], \n",
    "    sort=True, ascending=False).plot.barh(color=color)\n",
    "plt.title(f\"Number of Products per Category\")\n",
    "xlabels = [f'{x // 1000:,}K' for x in range(0, x_limit, x_step)]\n",
    "plt.xticks(range(0, x_limit, x_step), xlabels)\n",
    "plt.xlabel('Number of products')\n",
    "plt.setp(ax.get_xticklabels()[0], visible=False)\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 27,
   "id": "bbf5026d",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of bottom categories in train: 2,782'"
      ]
     },
     "execution_count": 27,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of bottom categories in train: {len(df_train['bottom_category_text'].unique()):,}\""
   ]
  },
  {
   "cell_type": "markdown",
   "id": "ec3b2b00",
   "metadata": {},
   "source": [
    "#### TFRecords"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 28,
   "id": "6a052026",
   "metadata": {},
   "outputs": [],
   "source": [
    "tfrecord_filename_train = f\"{PATH}/tfrecords/train/{tfrecord_filenames_train[1]}\"\n",
    "raw_dataset_train = tf.data.TFRecordDataset(tfrecord_filename_train)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 29,
   "id": "b15212f1",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a description of the features\n",
    "feature_desc = {\n",
    "    'product_id': tf.io.FixedLenFeature([], tf.int64),\n",
    "    'title': tf.io.FixedLenFeature([], tf.string),\n",
    "    'description': tf.io.FixedLenFeature([], tf.string),\n",
    "    'tags': tf.io.FixedLenFeature([], tf.string),\n",
    "    'image/encoded': tf.io.FixedLenFeature([], tf.string),\n",
    "    'image/width': tf.io.FixedLenFeature([], tf.int64),\n",
    "    'image/height': tf.io.FixedLenFeature([], tf.int64),\n",
    "    # only in training\n",
    "    'top_category_text': tf.io.FixedLenFeature([], tf.string),\n",
    "    'color_text': tf.io.FixedLenFeature([], tf.string),\n",
    "    # more can be added ...\n",
    "}"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 30,
   "id": "2f65f1a5",
   "metadata": {},
   "outputs": [],
   "source": [
    "def _parse_function(example_proto):\n",
    "    # Parse the input `tf.train.Example` proto using the dictionary above.\n",
    "    return tf.io.parse_single_example(example_proto, feature_desc)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 31,
   "id": "24753ee0",
   "metadata": {},
   "outputs": [],
   "source": [
    "parsed_dataset_train = raw_dataset_train.map(_parse_function)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 32,
   "id": "b695e95a",
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_images(parsed_dataset, column, N=10):\n",
    "    \n",
    "    images = []\n",
    "    titles = []\n",
    "    for features in parsed_dataset.take(N):\n",
    "        img_bytes = features['image/encoded'].numpy()\n",
    "        if column in features:\n",
    "            text = features[column].numpy().decode('utf-8')\n",
    "        else:\n",
    "            text = 'unknown'\n",
    "        bytes_io = io.BytesIO(img_bytes)\n",
    "        image = Image.open(bytes_io)\n",
    "        images.append(image)\n",
    "        titles.append(text)\n",
    "\n",
    "    return images, titles"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 33,
   "id": "39fc30a9",
   "metadata": {},
   "outputs": [],
   "source": [
    "images, titles = get_images(parsed_dataset_train, 'top_category_text')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 34,
   "id": "b1200c91",
   "metadata": {},
   "outputs": [],
   "source": [
    "def show(images, titles, N=100):\n",
    "    \n",
    "    num_cols = 5\n",
    "    num_rows = N // num_cols\n",
    "    plt_height = 3 * num_rows\n",
    "    \n",
    "    plt.figure(figsize=(12, plt_height))\n",
    "    for i, (image, title) in enumerate(zip(images[:N], titles[:N])):\n",
    "        ax = plt.subplot(num_rows, num_cols, i + 1)\n",
    "        plt.imshow(image)\n",
    "        plt.title(title, fontsize = 12)\n",
    "        plt.axis(\"off\")\n",
    "    plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 35,
   "id": "8e0bc308",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 1200x6000 with 10 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "show(images, titles)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0902ef9e",
   "metadata": {},
   "source": [
    "### Test Dataset\n",
    "\n",
    "The test dataset will be used to predict using the model/s you have trained\n",
    "\n",
    "**Note** that categories and colors for the test data are hidden, you have to predict them!\n",
    "\n",
    "#### Parquet"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 36,
   "id": "d1bf3eb7",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_test = get_parquet_data(\"test\", len(parquet_filenames_test))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 37,
   "id": "7f2a83db",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of rows in test: 27,119'"
      ]
     },
     "execution_count": 37,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f'Number of rows in test: {len(df_test):,}'"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 38,
   "id": "499136fb",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of unique products in test: 27,119'"
      ]
     },
     "execution_count": 38,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of unique products in test: {len(df_test['product_id'].unique()):,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 39,
   "id": "dcfa5a4b",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of columns in test: 15'"
      ]
     },
     "execution_count": 39,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of columns in test: {len(df_test.columns):,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 40,
   "id": "d6075cc6",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "['product_id',\n",
       " 'title',\n",
       " 'description',\n",
       " 'tags',\n",
       " 'type',\n",
       " 'room',\n",
       " 'craft_type',\n",
       " 'recipient',\n",
       " 'material',\n",
       " 'occasion',\n",
       " 'holiday',\n",
       " 'art_subject',\n",
       " 'style',\n",
       " 'shape',\n",
       " 'pattern']"
      ]
     },
     "execution_count": 40,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "list(df_test.columns)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 41,
   "id": "18d44252",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "product_id                                            1323824090\n",
       "title          12 Small Pressed Reddish-Orange Japanese Maple...\n",
       "description    12 small pressed reddish-orange Japanese maple...\n",
       "tags           Leaves,Art,Crafts,Paper crafts,Fall,Soap,Bookm...\n",
       "type                                                    physical\n",
       "room                                                        None\n",
       "craft_type                                      floral arranging\n",
       "recipient                                                   None\n",
       "material                                                    None\n",
       "occasion                                                    None\n",
       "holiday                                                     None\n",
       "art_subject                                                 None\n",
       "style                                                       None\n",
       "shape                                                       None\n",
       "pattern                                                     None\n",
       "Name: 0, dtype: object"
      ]
     },
     "execution_count": 41,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_test.iloc[0]"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "6506c24e",
   "metadata": {},
   "source": [
    "#### TFRecords"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 42,
   "id": "48fac509",
   "metadata": {},
   "outputs": [],
   "source": [
    "filename = f\"{PATH}/tfrecords/test/{tfrecord_filenames_test[2]}\"\n",
    "raw_dataset_test = tf.data.TFRecordDataset(filename)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 43,
   "id": "1aaee4bc",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "FixedLenFeature(shape=[], dtype=tf.string, default_value=None)"
      ]
     },
     "execution_count": 43,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "feature_desc.pop('top_category_text')\n",
    "feature_desc.pop('color_text')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 44,
   "id": "d710e5aa",
   "metadata": {},
   "outputs": [],
   "source": [
    "parsed_dataset_test = raw_dataset_test.map(_parse_function)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 45,
   "id": "a4988f39",
   "metadata": {},
   "outputs": [],
   "source": [
    "images, titles = get_images(parsed_dataset_test, 'top_category_text')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 46,
   "id": "fb768097",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 1200x6000 with 9 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "show(images, titles)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "b7a5336d",
   "metadata": {},
   "source": [
    "## Model Training\n",
    "\n",
    "Let's first split the data to be able to evaluate our model against a validation dataset. This validation set can also be used to fine tune your model before making the predictions on the hidden test set."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 47,
   "id": "f0c1a684",
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.model_selection import train_test_split\n",
    "\n",
    "df_training, df_validation = train_test_split(\n",
    "    df_train, \n",
    "    test_size=0.1, \n",
    "    stratify=df_train['bottom_category_text'], \n",
    "    random_state=42)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 48,
   "id": "5a8aedd5",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of products for training: 220,936'"
      ]
     },
     "execution_count": 48,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of products for training: {len(df_training):,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 49,
   "id": "fcd7b7ca",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of products for validation: 24,549'"
      ]
     },
     "execution_count": 49,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of products for validation: {len(df_validation):,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 50,
   "id": "bd371ead",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "electronics_and_accessories.car_parts_and_accessories.car_accessories.license_plates                                88\n",
       "clothing.womens_clothing.lingerie                                                                                   88\n",
       "clothing.gender_neutral_kids_clothing.hoodies_and_sweatshirts.sweatshirts                                           88\n",
       "accessories.hair_accessories.headbands.baby_headbands                                                               87\n",
       "pet_supplies.pet_furniture.play_furniture                                                                           87\n",
       "                                                                                                                    ..\n",
       "craft_supplies_and_tools.fabric_and_notions.notions.pins_and_needles.pin_and_needle_accessories.point_protectors    40\n",
       "shoes.mens_shoes.sneakers_and_athletic_shoes.platform_and_club_sneakers                                             39\n",
       "craft_supplies_and_tools.glue_and_adhesives.adhesives.adhesive_sealers_and_finishes                                 38\n",
       "toys_and_games.sports_and_outdoor_games.boating_and_water_sports                                                    38\n",
       "clothing.girls_clothing.underwear.petticoats                                                                        37\n",
       "Name: bottom_category_text, Length: 2782, dtype: int64"
      ]
     },
     "execution_count": 50,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_training['bottom_category_text'].value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 51,
   "id": "ce56c304",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "toys_and_games.toys.dolls_and_action_figures.doll_clothing.shirts          10\n",
       "paper_and_party_supplies.paper.stationery.programs                         10\n",
       "art_and_collectibles.prints.lithographs                                    10\n",
       "clothing.gender_neutral_kids_clothing.socks_and_leg_warmers.leg_warmers    10\n",
       "art_and_collectibles.painting.acrylic                                      10\n",
       "                                                                           ..\n",
       "toys_and_games.sports_and_outdoor_games.team_sports.gymnastics              4\n",
       "shoes.unisex_kids_shoes.clogs_and_mules                                     4\n",
       "shoes.mens_shoes.sneakers_and_athletic_shoes.platform_and_club_sneakers     4\n",
       "shoes.unisex_adult_shoes.boots.chukka_boots                                 4\n",
       "books_movies_and_music.music.picks_and_slides.slides                        4\n",
       "Name: bottom_category_text, Length: 2782, dtype: int64"
      ]
     },
     "execution_count": 51,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_validation['bottom_category_text'].value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 52,
   "id": "ef90659e",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "home_and_living                47879\n",
       "craft_supplies_and_tools       38110\n",
       "clothing                       28827\n",
       "toys_and_games                 12993\n",
       "accessories                    11345\n",
       "bath_and_beauty                11185\n",
       "paper_and_party_supplies        9727\n",
       "shoes                           9319\n",
       "weddings                        9301\n",
       "jewelry                         8508\n",
       "art_and_collectibles            7869\n",
       "electronics_and_accessories     7361\n",
       "books_movies_and_music          6639\n",
       "bags_and_purses                 5980\n",
       "pet_supplies                    5893\n",
       "Name: top_category_text, dtype: int64"
      ]
     },
     "execution_count": 52,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_training['top_category_text'].value_counts()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 53,
   "id": "538e13e5",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "home_and_living                5321\n",
       "craft_supplies_and_tools       4238\n",
       "clothing                       3201\n",
       "toys_and_games                 1434\n",
       "accessories                    1265\n",
       "bath_and_beauty                1241\n",
       "paper_and_party_supplies       1084\n",
       "shoes                          1036\n",
       "weddings                       1032\n",
       "jewelry                         949\n",
       "art_and_collectibles            872\n",
       "electronics_and_accessories     816\n",
       "books_movies_and_music          739\n",
       "bags_and_purses                 665\n",
       "pet_supplies                    656\n",
       "Name: top_category_text, dtype: int64"
      ]
     },
     "execution_count": 53,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "df_validation['top_category_text'].value_counts()"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "d07114d1",
   "metadata": {},
   "source": [
    "### Training\n",
    "\n",
    "As a simple example, let's follow the [Working With Text Data](https://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html) tutorial from scikit-learn that:\n",
    "\n",
    "- processes the text using `CountVectorizer`, that builds a dictionary of features and transforms documents to feature vectors\n",
    "- extracts term frequencies and inverse document frequencies using `TfidfTransformer`, (see [TF-IDF](https://en.wikipedia.org/wiki/Tf%E2%80%93idf))\n",
    "- trains a naïve Bayes classifier, which gives us a nice baseline"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 54,
   "id": "a03f88ad",
   "metadata": {},
   "outputs": [],
   "source": [
    "from sklearn.feature_extraction.text import CountVectorizer\n",
    "from sklearn.feature_extraction.text import TfidfTransformer\n",
    "from sklearn.naive_bayes import MultinomialNB\n",
    "from sklearn.pipeline import Pipeline\n",
    "\n",
    "text_clf = Pipeline([\n",
    "    ('vect', CountVectorizer()),\n",
    "    ('tfidf', TfidfTransformer()),\n",
    "    ('clf', MultinomialNB()),\n",
    "])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 55,
   "id": "554f19ce",
   "metadata": {},
   "outputs": [],
   "source": [
    "train_text = df_training['title'].fillna('').values"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 56,
   "id": "54501582",
   "metadata": {},
   "outputs": [],
   "source": [
    "train_labels = df_training['top_category_text'].values"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 57,
   "id": "4bf4d653",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/html": [
       "<style>#sk-container-id-1 {color: black;background-color: white;}#sk-container-id-1 pre{padding: 0;}#sk-container-id-1 div.sk-toggleable {background-color: white;}#sk-container-id-1 label.sk-toggleable__label {cursor: pointer;display: block;width: 100%;margin-bottom: 0;padding: 0.3em;box-sizing: border-box;text-align: center;}#sk-container-id-1 label.sk-toggleable__label-arrow:before {content: \"▸\";float: left;margin-right: 0.25em;color: #696969;}#sk-container-id-1 label.sk-toggleable__label-arrow:hover:before {color: black;}#sk-container-id-1 div.sk-estimator:hover label.sk-toggleable__label-arrow:before {color: black;}#sk-container-id-1 div.sk-toggleable__content {max-height: 0;max-width: 0;overflow: hidden;text-align: left;background-color: #f0f8ff;}#sk-container-id-1 div.sk-toggleable__content pre {margin: 0.2em;color: black;border-radius: 0.25em;background-color: #f0f8ff;}#sk-container-id-1 input.sk-toggleable__control:checked~div.sk-toggleable__content {max-height: 200px;max-width: 100%;overflow: auto;}#sk-container-id-1 input.sk-toggleable__control:checked~label.sk-toggleable__label-arrow:before {content: \"▾\";}#sk-container-id-1 div.sk-estimator input.sk-toggleable__control:checked~label.sk-toggleable__label {background-color: #d4ebff;}#sk-container-id-1 div.sk-label input.sk-toggleable__control:checked~label.sk-toggleable__label {background-color: #d4ebff;}#sk-container-id-1 input.sk-hidden--visually {border: 0;clip: rect(1px 1px 1px 1px);clip: rect(1px, 1px, 1px, 1px);height: 1px;margin: -1px;overflow: hidden;padding: 0;position: absolute;width: 1px;}#sk-container-id-1 div.sk-estimator {font-family: monospace;background-color: #f0f8ff;border: 1px dotted black;border-radius: 0.25em;box-sizing: border-box;margin-bottom: 0.5em;}#sk-container-id-1 div.sk-estimator:hover {background-color: #d4ebff;}#sk-container-id-1 div.sk-parallel-item::after {content: \"\";width: 100%;border-bottom: 1px solid gray;flex-grow: 1;}#sk-container-id-1 div.sk-label:hover label.sk-toggleable__label {background-color: #d4ebff;}#sk-container-id-1 div.sk-serial::before {content: \"\";position: absolute;border-left: 1px solid gray;box-sizing: border-box;top: 0;bottom: 0;left: 50%;z-index: 0;}#sk-container-id-1 div.sk-serial {display: flex;flex-direction: column;align-items: center;background-color: white;padding-right: 0.2em;padding-left: 0.2em;position: relative;}#sk-container-id-1 div.sk-item {position: relative;z-index: 1;}#sk-container-id-1 div.sk-parallel {display: flex;align-items: stretch;justify-content: center;background-color: white;position: relative;}#sk-container-id-1 div.sk-item::before, #sk-container-id-1 div.sk-parallel-item::before {content: \"\";position: absolute;border-left: 1px solid gray;box-sizing: border-box;top: 0;bottom: 0;left: 50%;z-index: -1;}#sk-container-id-1 div.sk-parallel-item {display: flex;flex-direction: column;z-index: 1;position: relative;background-color: white;}#sk-container-id-1 div.sk-parallel-item:first-child::after {align-self: flex-end;width: 50%;}#sk-container-id-1 div.sk-parallel-item:last-child::after {align-self: flex-start;width: 50%;}#sk-container-id-1 div.sk-parallel-item:only-child::after {width: 0;}#sk-container-id-1 div.sk-dashed-wrapped {border: 1px dashed gray;margin: 0 0.4em 0.5em 0.4em;box-sizing: border-box;padding-bottom: 0.4em;background-color: white;}#sk-container-id-1 div.sk-label label {font-family: monospace;font-weight: bold;display: inline-block;line-height: 1.2em;}#sk-container-id-1 div.sk-label-container {text-align: center;}#sk-container-id-1 div.sk-container {/* jupyter's `normalize.less` sets `[hidden] { display: none; }` but bootstrap.min.css set `[hidden] { display: none !important; }` so we also need the `!important` here to be able to override the default hidden behavior on the sphinx rendered scikit-learn.org. See: https://github.com/scikit-learn/scikit-learn/issues/21755 */display: inline-block !important;position: relative;}#sk-container-id-1 div.sk-text-repr-fallback {display: none;}</style><div id=\"sk-container-id-1\" class=\"sk-top-container\"><div class=\"sk-text-repr-fallback\"><pre>Pipeline(steps=[(&#x27;vect&#x27;, CountVectorizer()), (&#x27;tfidf&#x27;, TfidfTransformer()),\n",
       "                (&#x27;clf&#x27;, MultinomialNB())])</pre><b>In a Jupyter environment, please rerun this cell to show the HTML representation or trust the notebook. <br />On GitHub, the HTML representation is unable to render, please try loading this page with nbviewer.org.</b></div><div class=\"sk-container\" hidden><div class=\"sk-item sk-dashed-wrapped\"><div class=\"sk-label-container\"><div class=\"sk-label sk-toggleable\"><input class=\"sk-toggleable__control sk-hidden--visually\" id=\"sk-estimator-id-1\" type=\"checkbox\" ><label for=\"sk-estimator-id-1\" class=\"sk-toggleable__label sk-toggleable__label-arrow\">Pipeline</label><div class=\"sk-toggleable__content\"><pre>Pipeline(steps=[(&#x27;vect&#x27;, CountVectorizer()), (&#x27;tfidf&#x27;, TfidfTransformer()),\n",
       "                (&#x27;clf&#x27;, MultinomialNB())])</pre></div></div></div><div class=\"sk-serial\"><div class=\"sk-item\"><div class=\"sk-estimator sk-toggleable\"><input class=\"sk-toggleable__control sk-hidden--visually\" id=\"sk-estimator-id-2\" type=\"checkbox\" ><label for=\"sk-estimator-id-2\" class=\"sk-toggleable__label sk-toggleable__label-arrow\">CountVectorizer</label><div class=\"sk-toggleable__content\"><pre>CountVectorizer()</pre></div></div></div><div class=\"sk-item\"><div class=\"sk-estimator sk-toggleable\"><input class=\"sk-toggleable__control sk-hidden--visually\" id=\"sk-estimator-id-3\" type=\"checkbox\" ><label for=\"sk-estimator-id-3\" class=\"sk-toggleable__label sk-toggleable__label-arrow\">TfidfTransformer</label><div class=\"sk-toggleable__content\"><pre>TfidfTransformer()</pre></div></div></div><div class=\"sk-item\"><div class=\"sk-estimator sk-toggleable\"><input class=\"sk-toggleable__control sk-hidden--visually\" id=\"sk-estimator-id-4\" type=\"checkbox\" ><label for=\"sk-estimator-id-4\" class=\"sk-toggleable__label sk-toggleable__label-arrow\">MultinomialNB</label><div class=\"sk-toggleable__content\"><pre>MultinomialNB()</pre></div></div></div></div></div></div></div>"
      ],
      "text/plain": [
       "Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),\n",
       "                ('clf', MultinomialNB())])"
      ]
     },
     "execution_count": 57,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "text_clf.fit(train_text, train_labels)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "c4bfcad1",
   "metadata": {},
   "source": [
    "### Evaluation\n",
    "\n",
    "Using our validation dataset, compute a number of metrics to measure how well we generalize on our validation set."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 58,
   "id": "c8a0ab7d",
   "metadata": {},
   "outputs": [],
   "source": [
    "val_text = df_validation['title'].fillna('').values\n",
    "val_labels = df_validation['top_category_text'].values"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 59,
   "id": "4d9abd32",
   "metadata": {},
   "outputs": [],
   "source": [
    "predictions = text_clf.predict(val_text)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 60,
   "id": "48846387",
   "metadata": {},
   "outputs": [],
   "source": [
    "f1 = metrics.f1_score(val_labels, predictions, average='weighted')"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 61,
   "id": "efa8ff9c",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'F1 score: 0.6446'"
      ]
     },
     "execution_count": 61,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"F1 score: {f1:.4f}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 62,
   "id": "c568d196",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "                             precision    recall  f1-score   support\n",
      "\n",
      "                accessories       0.83      0.39      0.53      1265\n",
      "       art_and_collectibles       0.92      0.09      0.16       872\n",
      "            bags_and_purses       0.91      0.29      0.44       665\n",
      "            bath_and_beauty       0.91      0.51      0.65      1241\n",
      "     books_movies_and_music       0.96      0.40      0.56       739\n",
      "                   clothing       0.73      0.97      0.83      3201\n",
      "   craft_supplies_and_tools       0.59      0.84      0.70      4238\n",
      "electronics_and_accessories       0.97      0.48      0.64       816\n",
      "            home_and_living       0.56      0.94      0.70      5321\n",
      "                    jewelry       0.88      0.47      0.61       949\n",
      "   paper_and_party_supplies       0.85      0.47      0.60      1084\n",
      "               pet_supplies       0.96      0.26      0.41       656\n",
      "                      shoes       0.94      0.72      0.81      1036\n",
      "             toys_and_games       0.90      0.47      0.62      1434\n",
      "                   weddings       0.76      0.25      0.38      1032\n",
      "\n",
      "                   accuracy                           0.67     24549\n",
      "                  macro avg       0.84      0.50      0.58     24549\n",
      "               weighted avg       0.75      0.67      0.64     24549\n",
      "\n"
     ]
    }
   ],
   "source": [
    "print(metrics.classification_report(val_labels, predictions))"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 63,
   "id": "484f013b",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "array([[ 494,    0,    1,    7,    0,  298,  237,    0,  174,    3,    0,\n",
       "           0,    1,    4,   46],\n",
       "       [   2,   79,    0,    1,    3,   50,  209,    0,  508,    0,    3,\n",
       "           2,    0,   15,    0],\n",
       "       [   1,    0,  192,    6,    1,   36,  163,    0,  247,    3,    1,\n",
       "           0,    0,   13,    2],\n",
       "       [  10,    0,    5,  627,    0,   77,  206,    1,  308,    2,    0,\n",
       "           0,    0,    4,    1],\n",
       "       [   0,    0,    0,    2,  293,   14,  138,    5,  270,    0,    8,\n",
       "           0,    0,    8,    1],\n",
       "       [   7,    0,    0,    3,    0, 3090,   30,    0,   64,    0,    0,\n",
       "           0,    4,    1,    2],\n",
       "       [   8,    5,    1,    7,    0,   32, 3550,    2,  598,    3,   13,\n",
       "           0,   10,    9,    0],\n",
       "       [   2,    0,    4,    0,    3,   11,  148,  392,  247,    0,    6,\n",
       "           1,    0,    2,    0],\n",
       "       [   1,    1,    2,   10,    2,   51,  242,    2, 4980,    6,   10,\n",
       "           1,    1,    8,    4],\n",
       "       [   9,    0,    0,    2,    0,   16,  352,    0,  114,  448,    0,\n",
       "           2,    1,    1,    4],\n",
       "       [   0,    1,    0,    0,    0,   31,  231,    2,  286,    0,  506,\n",
       "           0,    0,    5,   22],\n",
       "       [   0,    0,    0,   14,    0,   99,   60,    0,  299,    1,    1,\n",
       "         172,    4,    6,    0],\n",
       "       [   2,    0,    0,    0,    0,  170,   64,    0,   54,    0,    0,\n",
       "           0,  745,    1,    0],\n",
       "       [   9,    0,    1,    2,    4,  133,  192,    0,  403,    2,    4,\n",
       "           2,    1,  679,    2],\n",
       "       [  53,    0,    4,    5,    0,  152,  153,    1,  292,   40,   41,\n",
       "           0,   27,    1,  263]])"
      ]
     },
     "execution_count": 63,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "metrics.confusion_matrix(val_labels, predictions)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "4f429dd7",
   "metadata": {},
   "source": [
    "**Note** that this is just a baseline that uses a naïve Bayes classifier and uses the `title` only as inputs, there are many more parameters that can be used in addition to multiple architectures and processing approaches"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "d89f941e",
   "metadata": {},
   "source": [
    "### Training\n",
    "\n",
    "As another example, let's train an image classification model to predict color based only on the image as input. We leveraged `InceptionV3` as the base model with `imagenet` weights and freezing the layers. As we are using the TFRecord dataset, we split the filenames into train and validation. \n",
    "\n",
    "You might want to find a more stratified way to split between train and validation."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 64,
   "id": "335a5747",
   "metadata": {},
   "outputs": [],
   "source": [
    "tf_training_filenames, tf_validation_filenames = train_test_split(\n",
    "    tf.io.gfile.glob(f\"{PATH}/tfrecords/train/part-*.tfrecord\"),\n",
    "    test_size=0.2,\n",
    "    random_state=42,\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 65,
   "id": "4dcf893e",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of TFRecords for training: 222'"
      ]
     },
     "execution_count": 65,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of TFRecords for training: {len(tf_training_filenames):,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 66,
   "id": "6daa5da0",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "'Number of TFRecords for training: 56'"
      ]
     },
     "execution_count": 66,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "f\"Number of TFRecords for training: {len(tf_validation_filenames):,}\""
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 67,
   "id": "987434ed",
   "metadata": {},
   "outputs": [],
   "source": [
    "# Create a description of the features.\n",
    "feature_spec = {\n",
    "    'product_id': tf.io.FixedLenFeature([], tf.int64),\n",
    "    'image/encoded': tf.io.FixedLenFeature([], tf.string),\n",
    "    'image/width': tf.io.FixedLenFeature([], tf.int64),\n",
    "    'image/height': tf.io.FixedLenFeature([], tf.int64),\n",
    "    'color_id': tf.io.FixedLenFeature([], tf.int64),\n",
    "}\n",
    "\n",
    "def parse_dataset(example_proto, feature_spec, num_channels):\n",
    "    \n",
    "    features = tf.io.parse_single_example(example_proto, feature_spec)\n",
    "    image_encoded = features[\"image/encoded\"]\n",
    "    img_width = features[\"image/width\"]\n",
    "    img_height = features[\"image/height\"]\n",
    "    color_id = features[\"color_id\"]\n",
    "\n",
    "    image = tf.reshape(\n",
    "        tf.io.decode_jpeg(image_encoded),\n",
    "        [img_height, img_width, num_channels],\n",
    "    )\n",
    "    image = tf.cast(image, tf.float32)\n",
    "    label = tf.cast(tf.one_hot(color_id, NUM_COLORS), tf.int64)\n",
    "\n",
    "    return (\n",
    "        { \"image\": image, },\n",
    "        label\n",
    "    )"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 68,
   "id": "b96ac16f",
   "metadata": {},
   "outputs": [],
   "source": [
    "def create_preprocessing_model(target_image_size, should_rescale):\n",
    "    preprocessing_layers = []\n",
    "    if should_rescale:\n",
    "        preprocessing_layers.append(\n",
    "            tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)\n",
    "        )\n",
    "    preprocessing_layers.append(\n",
    "        tf.keras.layers.experimental.preprocessing.Resizing(\n",
    "            target_image_size, target_image_size\n",
    "        )\n",
    "    )\n",
    "    return tf.keras.Sequential(preprocessing_layers)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 69,
   "id": "bc5c5c99",
   "metadata": {},
   "outputs": [],
   "source": [
    "def transform_image_data(example, preprocessing_model):\n",
    "    image = tf.expand_dims(example[\"image\"], 0)\n",
    "    example[\"image\"] = preprocessing_model(image)[0]\n",
    "    return example"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 70,
   "id": "5bfeec8c",
   "metadata": {},
   "outputs": [],
   "source": [
    "def get_dataset_batch(\n",
    "    file_pattern: [],\n",
    "    batch_size: int,\n",
    "    num_channels: int,\n",
    "    mode: tf.estimator.ModeKeys,\n",
    ") -> tf.data.Dataset:\n",
    "    \n",
    "    augment = shuffle = mode == tf.estimator.ModeKeys.TRAIN\n",
    "\n",
    "    dataset = tf.data.Dataset.list_files(\n",
    "        file_pattern, shuffle=shuffle, seed=42\n",
    "    )\n",
    "    dataset = dataset.interleave(\n",
    "        tf.data.TFRecordDataset, num_parallel_calls=tf.data.experimental.AUTOTUNE\n",
    "    )\n",
    "    dataset = dataset.map(\n",
    "        lambda x: parse_dataset(x, feature_spec, num_channels),\n",
    "        num_parallel_calls=tf.data.experimental.AUTOTUNE,\n",
    "    )\n",
    "    preprocessing_model = create_preprocessing_model(\n",
    "        target_image_size=224,\n",
    "        should_rescale=True,\n",
    "    )\n",
    "    dataset = dataset.map(\n",
    "        lambda x, y: (transform_image_data(x, preprocessing_model), y),\n",
    "        num_parallel_calls=tf.data.experimental.AUTOTUNE,\n",
    "    )\n",
    "    dataset = dataset.batch(batch_size, drop_remainder=True)\n",
    "    dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)\n",
    "\n",
    "    return dataset"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 71,
   "id": "c0a9e81d",
   "metadata": {},
   "outputs": [],
   "source": [
    "BATCH_SIZE = 128"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 72,
   "id": "8c0ff734",
   "metadata": {
    "scrolled": false
   },
   "outputs": [],
   "source": [
    "train_dataset = get_dataset_batch(\n",
    "    file_pattern=tf_training_filenames,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    num_channels=3,\n",
    "    mode=tf.estimator.ModeKeys.TRAIN,\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 73,
   "id": "dd0be60c",
   "metadata": {},
   "outputs": [],
   "source": [
    "val_dataset = get_dataset_batch(\n",
    "    file_pattern=tf_validation_filenames,\n",
    "    batch_size=BATCH_SIZE,\n",
    "    num_channels=3,\n",
    "    mode=tf.estimator.ModeKeys.EVAL,\n",
    ")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 74,
   "id": "16a83000",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "text/plain": [
       "[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),\n",
       " PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]"
      ]
     },
     "execution_count": 74,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "gpus = tf.config.list_physical_devices(\"GPU\")\n",
    "gpus"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 75,
   "id": "37828cc0",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')\n"
     ]
    }
   ],
   "source": [
    "strategy = tf.distribute.MirroredStrategy()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 76,
   "id": "f1c9b0ef",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).\n",
      "INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).\n",
      "INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).\n",
      "INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).\n",
      "INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).\n",
      "INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).\n",
      "INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).\n",
      "INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).\n",
      "INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).\n",
      "INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).\n"
     ]
    }
   ],
   "source": [
    "with strategy.scope():\n",
    "    \n",
    "    input_shape = (224, 224, 3)\n",
    "    inputs = tf.keras.layers.Input(shape=input_shape, dtype=tf.float32, name=\"image\")\n",
    "    base_model = InceptionV3(weights='imagenet', include_top=False, input_tensor=inputs)\n",
    "    x = base_model.output\n",
    "    x = GlobalAveragePooling2D()(x)\n",
    "    x = Dense(1024, activation='relu')(x)\n",
    "    predictions = Dense(NUM_COLORS, activation='softmax')(x)\n",
    "    \n",
    "    model = Model(inputs=base_model.input, outputs=predictions)\n",
    "    \n",
    "    model.compile(loss=\"categorical_crossentropy\", optimizer=\"adam\", metrics=[\"accuracy\"])"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 77,
   "id": "4b7b1aa5",
   "metadata": {},
   "outputs": [],
   "source": [
    "for layer in base_model.layers:\n",
    "    layer.trainable = False"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 78,
   "id": "c35029b9",
   "metadata": {},
   "outputs": [],
   "source": [
    "# model.summary()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 79,
   "id": "3594687a",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "Epoch 1/3\n",
      "INFO:tensorflow:batch_all_reduce: 192 all-reduces with algorithm = nccl, num_packs = 1\n",
      "INFO:tensorflow:batch_all_reduce: 192 all-reduces with algorithm = nccl, num_packs = 1\n",
      "1519/1519 [==============================] - 831s 527ms/step - loss: 2.1454 - accuracy: 0.3650 - val_loss: 6.6118 - val_accuracy: 0.3339\n",
      "Epoch 2/3\n",
      "1519/1519 [==============================] - 793s 522ms/step - loss: 1.9508 - accuracy: 0.4193 - val_loss: 1.8446 - val_accuracy: 0.4484\n",
      "Epoch 3/3\n",
      "1519/1519 [==============================] - 792s 521ms/step - loss: 1.8720 - accuracy: 0.4410 - val_loss: 1.8921 - val_accuracy: 0.4406\n"
     ]
    }
   ],
   "source": [
    "epochs = 3\n",
    "\n",
    "hist = model.fit(\n",
    "    train_dataset,\n",
    "    epochs=epochs,\n",
    "    validation_data=val_dataset,\n",
    "    verbose=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 80,
   "id": "bb0209be",
   "metadata": {},
   "outputs": [
    {
     "data": {
      "image/png": "...\n",
      "text/plain": [
       "<Figure size 640x480 with 1 Axes>"
      ]
     },
     "metadata": {},
     "output_type": "display_data"
    }
   ],
   "source": [
    "plt.plot(hist.history[\"accuracy\"])\n",
    "plt.plot(hist.history[\"val_accuracy\"])\n",
    "plt.title(\"model accuracy\")\n",
    "plt.ylabel(\"accuracy\")\n",
    "plt.xlabel(\"epoch\")\n",
    "plt.legend([\"train\", \"validation\"], loc=\"upper left\")\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 81,
   "id": "8dfcd445",
   "metadata": {},
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "398/398 [==============================] - 52s 131ms/step - loss: 1.8921 - accuracy: 0.4406\n"
     ]
    },
    {
     "data": {
      "text/plain": [
       "[1.892129898071289, 0.44064071774482727]"
      ]
     },
     "execution_count": 81,
     "metadata": {},
     "output_type": "execute_result"
    }
   ],
   "source": [
    "model.evaluate(val_dataset, batch_size=BATCH_SIZE, verbose=1)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 82,
   "id": "569a452f",
   "metadata": {
    "scrolled": false
   },
   "outputs": [
    {
     "name": "stdout",
     "output_type": "stream",
     "text": [
      "398/398 [==============================] - 55s 127ms/step\n"
     ]
    }
   ],
   "source": [
    "predictions = model.predict(val_dataset, batch_size=BATCH_SIZE, verbose=1)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "fca67e94",
   "metadata": {},
   "source": [
    "**Note** again that is just another baseline that only uses the `image` as the input and generates predictions after freezing all the layers for that base model.\n",
    "\n",
    "Go wild and try different architectures and approaches, use several modalities, apply augmentations on the preprocessing layers, predict several targets at the same time, etc!"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "5ff1e03c",
   "metadata": {},
   "source": [
    "## Submission\n",
    "\n",
    "Prepare a submission for the test data. Here I am using some dummy values, all zeros, as the ids for top categories, bottom categories and colors. Feed-forward the test data through your model, get the predictions per product and output them onto a file to be checked against the true values.\n",
    "\n",
    "Happy Hacking!"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 83,
   "id": "7f4ae730",
   "metadata": {},
   "outputs": [],
   "source": [
    "df_predictions = df_test.copy()"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 84,
   "id": "2491d9cc",
   "metadata": {},
   "outputs": [],
   "source": [
    "# dummy values\n",
    "df_predictions[\"top_category_id\"] = np.zeros(len(df_test), dtype=int)\n",
    "df_predictions[\"bottom_category_id\"] = np.zeros(len(df_test), dtype=int)\n",
    "df_predictions[\"color_id\"] = np.zeros(len(df_test), dtype=int)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 85,
   "id": "86c2afbb",
   "metadata": {},
   "outputs": [],
   "source": [
    "student_id = \"12345689\"\n",
    "df_predictions[\n",
    "    [\"product_id\", \"top_category_id\", \"bottom_category_id\", \"color_id\"]\n",
    "].to_parquet(f\"predictions_{student_id}.parquet\")"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "85f73b7c",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.7.10"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}

NameError: name 'false' is not defined